In [1]:
# ! pip install python_hiccup

In [2]:
# ! pip install edn_format

In [3]:
from functools import partial

def walk(inner, outer, coll):
    if isinstance(coll, list):
        return outer([inner(e) for e in coll])
    elif isinstance(coll, dict):
        return outer(dict([inner(e) for e in iter(coll.items())]))
    elif isinstance(coll, tuple):
        return outer(tuple([inner(e) for e in coll]))
    else:
        return outer(coll)

def identity(e):
    return e
    
def prewalk(fn, coll):
    return walk(partial(prewalk, fn), identity, fn(coll))

def postwalk(fn, coll):
    return walk(partial(postwalk, fn), fn, coll)

In [4]:
import edn_format

def ednTypesToPy(edn):
    if isinstance(edn, edn_format.immutable_list.ImmutableList):
        return list(edn)
    elif isinstance(edn, edn_format.immutable_dict.ImmutableDict):
        return dict(edn)
    elif isinstance(edn, edn_format.edn_lex.Keyword):
        return str(edn)[1:]
    else:
        return edn

def execute(coll):
    if isinstance(coll, tuple) and isinstance(coll[0],edn_format.edn_lex.Symbol):
        return globals()[str(coll[0])](*coll[1:])
    else:
        return coll

In [5]:
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.display import display, HTML
from python_hiccup.html import render

@magics_class
class MyMagics(Magics):
    @cell_magic
    def hiccup(self, line, cell):
        display(HTML(render(postwalk(execute,prewalk(ednTypesToPy,edn_format.loads(cell))))))
        return None

In [6]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

In [7]:
%%hiccup
[:h1 "Hello world!"]

In [8]:
%%hiccup
[:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:red"}]]

In [9]:
def subs(s, m):
    return s[m:]

def plus(a, b):
    return a + b

In [10]:
%%hiccup
[:h1 (subs (subs "hiabc" (plus 1 1)) 1)]